In [ ]:
#TESTING WEB SCRAPING

#For single recipe, taking nutrient info

In [1]:
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd

In [ ]:
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
url1 = url + 'fullrecipenutrition'
r = requests.get(url1)
print('Response body: {0}'.format(r.text))

In [ ]:
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

In [ ]:
#this is the div containing all the nutrition information
nutrition_info = soup.find_all('div', class_='nutrition-row')

In [ ]:
#nutrition_info

In [ ]:
nutrient_list = []
recipe = soup.find('h2').text
for n in nutrition_info:
    name = n.find(class_ = 'nutrient-name').text
    amount = n.find(class_ = 'nutrient-value').text
    name = name[:name.index(':')]
    nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
nutrients = pd.DataFrame(nutrient_list)

nutrients.head()

In [ ]:
# & ingredients

In [ ]:
s = requests.get(url)

In [ ]:
page_body2 = s.text
soup2 = BeautifulSoup(page_body2, 'html.parser')

In [ ]:
ingredient_info = soup2.find_all(class_='recipe-container-outer')
#ingredient_info

In [ ]:
ingredient_list = []
for i in ingredient_info:
    ingredients = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
    for x in ingredients:
        ingredient_list.append({'ingredient': x.text, 'URL':url})

ingredients = pd.DataFrame(ingredient_list)

ingredients.head()

In [16]:
# FUNCTION FOR WEB SCRAPING

import time
def get_recipe_data(url, nutrient_df, ingredient_df):
    """
    Scrape allrecipes.com for information on the nutrient content and ingredients of each recipe
    Based on the url, ingredients and nutrient content are extracted separately from the website.
    
    IN: url: URL of the specific recipe as a string
    OUT: nutrient and ingredient dataframes with the information for the given recipe
    """
    
    #nutrition information
    url_nutrition = url + 'fullrecipenutrition'
    time.sleep(1)
    r1 = requests.get(url_nutrition)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    
    #this is the div containing all the nutrition information
    nutrition_info = soup1.find_all('div', class_='nutrition-row')
    
    nutrient_list = []
    recipe = soup1.find('h2').text
    for n in nutrition_info:
        name = n.find(class_ = 'nutrient-name').text
        amount = n.find(class_ = 'nutrient-value').text
        name = name[:name.index(':')]
        nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
    
    nutrients = pd.DataFrame(nutrient_list)
    nutrient_df = nutrient_df.append(nutrients, sort=True)
    
    #ingredients
    r2 = requests.get(url)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    ingredient_info = soup2.find_all(class_='recipe-container-outer')
    
    ingredient_list = []
    for i in ingredient_info:
        ingredient = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
        for x in ingredient:
            ingredient_list.append({'ingredient': x.text, 'URL':url})

    ingredients = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.append(ingredients, sort=True)
    
    return nutrient_df, ingredient_df

In [ ]:
#test the function
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
nutrient_columns = ['URL', 'amount', 'nutrient', 'recipe']
ingredient_columns = ['URL', 'ingredient']
nutrient_df = pd.DataFrame(columns=nutrient_columns)
ingredient_df = pd.DataFrame(columns=ingredient_columns)

nutrient_df, ingredient_df = get_recipe_data(url, nutrient_df, ingredient_df)

In [ ]:
#url2 = 'https://www.allrecipes.com/recipe/7589/allspice-cream-cheese-frosting/'
#nutrient_df, ingredient_df = get_recipe_data(url2, nutrient_df, ingredient_df)

In [ ]:
#nutrient_df

In [10]:
dinner_url_df = pd.read_csv('data/recipeLists/dinner_url.csv')
dinner_url_df.columns = ["originGridUrl", "recipeName", "recipeURL"]
#pd.options.display.max_colwidth = 200
#display(dinner_url_df['recipeURL'].head())

In [11]:
#dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].split("?")[0]
dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].astype(str).apply(lambda x: x.split("?")[0])
#dinner_url_df['recipeURLPrime'] = str(dinner_url_df['recipeURL']).split("?")[0]
pd.set_option('display.max_rows', None,'display.max_columns', None)
#display(dinner_url_df.head())

In [6]:
dinnerURLList = dinner_url_df.recipeURL.tolist()

In [21]:
dinnerURLList[:5]

['https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/',
 'https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/',
 'https://www.allrecipes.com/recipe/23847/pasta-pomodoro/',
 'https://www.allrecipes.com/recipe/50435/fry-bread-tacos-ii/',
 'https://www.allrecipes.com/recipe/140829/pork-marsala/']

In [31]:
nutrient_columns = ['URL', 'amount', 'nutrient', 'recipe']
ingredient_columns = ['URL', 'ingredient']
nutrient_df = pd.DataFrame(columns=nutrient_columns)
ingredient_df = pd.DataFrame(columns=ingredient_columns)

for url in dinnerURLList:
    nutrient_df, ingredient_df = get_recipe_data(url, nutrient_df, ingredient_df)
    time.sleep(1)
    

AttributeError: 'NoneType' object has no attribute 'text'

In [32]:
nutrient_df

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,20.1g,Total Fat,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,8.0g,Saturated Fat,Smothered Chicken Breasts
2,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,124mg,Cholesterol,Smothered Chicken Breasts
3,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,809mg,Sodium,Smothered Chicken Breasts
4,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,377mg,Potassium,Smothered Chicken Breasts
5,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,16.9g,Total Carbohydrates,Smothered Chicken Breasts
6,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,0.5g,Dietary Fiber,Smothered Chicken Breasts
7,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,44g,Protein,Smothered Chicken Breasts
8,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,15g,Sugars,Smothered Chicken Breasts
9,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/,29IU,Vitamin A,Smothered Chicken Breasts


In [33]:
nutrient_df.shape

(1656, 4)

In [37]:
dinnerURLList

['https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/',
 'https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/',
 'https://www.allrecipes.com/recipe/23847/pasta-pomodoro/',
 'https://www.allrecipes.com/recipe/50435/fry-bread-tacos-ii/',
 'https://www.allrecipes.com/recipe/140829/pork-marsala/',
 'https://www.allrecipes.com/recipe/217981/grilled-chicken-with-rosemary-and-bacon/',
 'https://www.allrecipes.com/recipe/87624/beefy-baked-ravioli/',
 'https://www.allrecipes.com/recipe/87648/fresh-tomato-shrimp-pasta/',
 'https://www.allrecipes.com/recipe/80388/lemon-rosemary-salmon/',
 'https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/',
 'https://www.allrecipes.com/recipe/50644/broiled-tilapia-parmesan/',
 'https://www.allrecipes.com/recipe/16066/awesome-slow-cooker-pot-roast/',
 'https://www.allrecipes.com/recipe/25203/brown-sugar-meatloaf/',
 'https://www.allrecipes.com/recipe/11758/baked-ziti-i/',
 'https://www.allrecipes.com/recipe/9023/baked-teri